import packages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

create a beautiful soup object

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html.parser')

get data into a list

In [3]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span == None:
        pass
    elif row.span.text.split("(")[0] =='Y':
        pass
    else: 
        cell['Borough'] = row.span.text.split("/")[2].split("(")[1].strip(')')
        cell['Latitude'] = float(row.span.text.split("/")[2].split("(")[0][:8])
        cell['Longitude'] = row.span.text.split("/")[2].split(";")[1].split(' ')[1][:-1]
        table_contents.append(cell)


convert data into a dataframe

In [4]:
df = pd.DataFrame(table_contents)
new_row = {'Borough': 'City of London', 'Latitude': 51.5155, 'Longitude': -0.0922 }
df_london = df.append(new_row, ignore_index = True)

In [5]:

df_london

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198
5,Camden,51.5290,-0.1255
6,Croydon,51.3714,-0.0977
7,Ealing,51.5130,-0.3089
8,Enfield,51.6538,-0.0799
9,Greenwich,51.4892,0.0648


In [6]:
df_london.shape

(33, 3)

## Define foursquare credential

In [7]:
CLIENT_ID = 'PTBQE2DOOJIFNTKH5YLSOCAWN1AS3EWCINOX0BUH30FH5E0V' # Foursquare ID
CLIENT_SECRET = 'IRVTNCASERYLWAVUBRMEO1AAYQAIG4VK13U5FU2CWZVSZLSZ' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: PTBQE2DOOJIFNTKH5YLSOCAWN1AS3EWCINOX0BUH30FH5E0V
CLIENT_SECRET:IRVTNCASERYLWAVUBRMEO1AAYQAIG4VK13U5FU2CWZVSZLSZ


## 

## Create a function to get venues

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Get venue

In [9]:
Lon_venues = getNearbyVenues(names=df_london['Borough'],
                                   latitudes=df_london['Latitude'],
                                   longitudes=df_london['Longitude'])


Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster
City of London


Count number of venues in each neighborhood

In [10]:
London = Lon_venues.groupby('Neighborhood').count().reset_index()

In [11]:
London.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,15,15,15,15,15,15
1,Barnet,9,9,9,9,9,9
2,Bexley,49,49,49,49,49,49
3,Brent,88,88,88,88,88,88
4,Bromley,60,60,60,60,60,60


Load airbnb listing data

In [12]:
# The code was removed by Watson Studio for sharing.

In [13]:
listing.head()

,neighbourhood_cleansed,accommodates,price
0,Lambeth,4,110.0
1,Islington,2,40.0
2,Kensington and Chelsea,2,75.0
3,Westminster,6,307.0
4,Hammersmith and Fulham,2,150.0


Calculate price per bed

In [14]:
listing['price_per_bed'] = listing['price'] / listing['accommodates']

In [15]:
listing.head()

,neighbourhood_cleansed,accommodates,price,price_per_bed
0,Lambeth,4,110.0,27.500000
1,Islington,2,40.0,20.000000
2,Kensington and Chelsea,2,75.0,37.500000
3,Westminster,6,307.0,51.166667
4,Hammersmith and Fulham,2,150.0,75.000000


In [16]:
listing = listing.groupby('neighbourhood_cleansed').mean().reset_index()
listing.rename(columns={"neighbourhood_cleansed":"Neighborhood"}, inplace=True)

In [17]:
listing.columns


Index(['Neighborhood', 'accommodates', 'price', 'price_per_bed'], dtype='object')

Combine listing and venue files

In [18]:
London_merged = London.merge(listing.set_index('Neighborhood'), on='Neighborhood')


In [19]:
London_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,accommodates,price,price_per_bed
0,Barking and Dagenham,15,15,15,15,15,15,2.606516,63.922306,30.109057
1,Barnet,9,9,9,9,9,9,2.939450,100.646483,48.427835
2,Bexley,49,49,49,49,49,49,2.458484,57.339350,25.657597
3,Brent,88,88,88,88,88,88,2.984449,97.952484,36.348829
4,Bromley,60,60,60,60,60,60,2.701599,72.701599,29.612419


In [20]:
London_merged.drop(columns=['Neighborhood Latitude' , 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], inplace=True)

London_merged.head()

,Neighborhood,Venue,accommodates,price,price_per_bed
0,Barking and Dagenham,15,2.606516,63.922306,30.109057
1,Barnet,9,2.939450,100.646483,48.427835
2,Bexley,49,2.458484,57.339350,25.657597
3,Brent,88,2.984449,97.952484,36.348829
4,Bromley,60,2.701599,72.701599,29.612419


In [21]:
London_sorted = London_merged.sort_values(['price_per_bed', 'Venue'], ascending=[False, False])
London_sorted.head(10)

,Neighborhood,Venue,accommodates,price,price_per_bed
32,Westminster,100,3.390317,186.926001,58.858879
19,Kensington and Chelsea,100,3.420190,181.939429,53.871180
6,City of London,100,3.066986,157.722488,53.770455
1,Barnet,9,2.939450,100.646483,48.427835
18,Islington,100,2.997682,135.591331,47.456859
5,Camden,100,3.182067,133.259035,46.974380
23,Merton,30,2.990909,107.325620,41.267208
12,Hammersmith and Fulham,100,3.243267,122.923009,39.996121
27,Southwark,100,2.971787,105.307933,38.845497
8,Ealing,100,2.778046,96.381182,38.694530
